In [72]:
import simpy
import pandas as pd
import numpy as np 
from collections import namedtuple
#from recordtype import recordtype
import random 
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from prettytable import PrettyTable
#get_ipython().magic('matplotlib inline')
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam 
from collections import deque 
from itertools import product
import math

class DQN:#모델 선언
    def __init__(self): #parameter들의 초기값
        #self.gamma = 0.85
        self.epsilon = 0.999
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.95
        self.tau = 0.125 #?
        self.learning_rate = 1.0
        self.memory = deque(maxlen= 2000)
        self.model = self.create_model()
        self.target_model = self.create_model() #target_model이란 create_model의 return값..?



    # create the neural network to train the q function 
    def create_model(self): #Q값예측모델. input:state 1개 output:action 10개?
        model = Sequential()
#         st = np.array(state).reshape(1,len(state))#1차원 리스트로 reshape한 state_space
#         #act = np.array(action).reshape(1,len(action))
#         state_shape = st.shape
#         action_shape = act.shape
        model.add(Dense(24, input_dim= 3, activation= 'relu')) # input은 state
        model.add(Dense(48, activation= 'relu'))
        model.add(Dense(24, activation= 'relu'))
        model.add(Dense(63)) #계산했을 때,(1~10)까지 세 수의 합이 10이 되는 경우의수는 63개. output에 대한 가중치는 매번 update되기 때문에 이에 mapping시키면 된다. 
        model.compile(loss= 'mean_squared_error', optimizer= Adam(lr= self.learning_rate))
        
        return model 



    # Action function to choose the best action given the q-function if not exploring based on epsilon p값에 의한 예측이 아닐때
    def choose_action(self, state, allowed_actions): #action을 선택 (parameter로 선택가능한 action이 들어옴)
        #각 source별 allowed action이 매우 많은데 output은 source별 q값인지? 아니면 각 가능한 action별 q값인지?
        
        self.epsilon *= self.epsilon_decay
        self.epsilon = max(self.epsilon_min, self.epsilon)
        r = np.random.random()
        if r < self.epsilon: #p값보다 작은 경우 랜덤한 액션을 취함
            #print("**** Choosing a random action ****")
#             for a in range(len(allowed_actions)) :
#                 rnd=random.choice(allowed_actions)
#                 if ((rnd[0]>=state[0][0])and(rnd[1]>=state[0][1])and(rnd[2]>=state[0][2])):
#                     break
            for i in range(len(allowed_actions)):
                rnd=random.choice(allowed_actions)
                
                if ((rnd[0]>=state[0])and(rnd[1]>=state[1])and(rnd[2]>=state[2])):
                    break                         
                                        
            return rnd #가능한 action중 랜덤하게 선택
        
        state = np.array(state).reshape(1,len(state)) #p값보다 큰경우, state 배열 생성

        pred = self.model.predict(state)[0]
        
        #state=0이면 filesize=0이라는 뜻인데 이 때 action을 0으로 할당해줘야 함. 만약 1개의 flow가 완료된경우는 어떻게 분배할 지 애매 
        
        return self.maxQ_action(pred,state,allowed_actions) #Q예측값중 min_rate 이상으로 가장 큰 action을 선택

    def maxQ_action(self,pred,state,allowed_actions):#allowed action 생성 (min_rate 이상 조합만 남김)
   
        #for p in range(len(pred)):
#             index=np.argmax(pred)
#             if ((allowed_actions[index][0]>=state[0][0])and(allowed_actions[index][1]>=state[0][1])and(allowed_actions[index][2]>=state[0][2])):#모든 action이 min_rate보다 크거나 같으면
#                 #print("선택된 action set은:",allowed_actions[index])
#                 return allowed_actions[index]
#             else:
#                 pred[index]=np.min(pred)
                
        #만약 return되지 못한 경우(즉, pred값이 가장 큰 index가 min_rate 조건을 충족시키지 못할 경우) 
        #위 코드를 주석처리하면 min_rate에 신경쓰지 않고 action을 할당하게 되는 코드
        return allowed_actions[np.argmax(pred)]
        
        
        
    # create replay buffer memory to sample randomly #메모리에서 꺼내서 학습할 수 있게 저장
    def remember(self, state, action, reward, next_state):
        print("record states")
        self.memory.append([state, action, reward, next_state])


    # build the replay buffer 저장한 것을 버퍼에서 꺼내오는.? 학습단계?
    def replay(self,allowed_actions):
        print ("train")
        
        batch_size = 32
        #list_of_next_allowed_machines = []
        if len(self.memory) < batch_size: #buffer에 저장된 memory가 buffer의 총 batch_size보다 작다면 return
            return 
        samples = random.sample(self.memory, batch_size) #메모리에서 배치사이즈만큼 랜덤으로 선택
        for sample in samples:
            state, action, reward, new_state = sample # sample 데이터 하나를 꺼내서
            state = np.array(state).reshape(1,state.shape[0]) 
            new_state = np.array(new_state).reshape(1,new_state.shape[0])
            target = self.target_model.predict(state) #기존 state로 target을 예측
            
            action_id = allowed_actions.index(action) #63개의 allowed action 중에서 state에 대한 action의 index를 추출
#             if done: #전체 queue를 모두 끝냈다면
#                 target[0][action_id] = reward #done이면 reward를 줌
#             else:
#                 # take max only from next_allowed_actions
#                 for i in range(len(new_action)):
#                     list_of_next_allowed_machines.append(int(new_action[i].name[-1]))

            target[0][action_id]=reward #63개의 action을 예측한 target에서 state에 대한 실제 action에 reward를 할당
                
            
            
            next_pred = self.target_model.predict(new_state)[0] #new state에 대한 target 예측
            
#             A=list(range(new_state[0],11))
#             B=list(range(new_state[1],11))
#             C=list(range(new_state[2],11))
#             ls=list(product(*[A,B,C]))
#             s=[]
        
        
#             for l in ls:
#                 #print (l)
#                 if ((l[0]+l[1]+l[2])<=10):
#                     s.append(x[l[0]]+y[l[1]]+z[l[2]])
            new_action=self.maxQ_action(next_pred,new_state,allowed_actions) # new state에 대한 action예측
            
            Q_future=next_pred[allowed_actions.index(new_action)] #next state에 대한 action의 Q값은?
                
#                 next_pred = self.target_model.predict(new_state)[0] #new state에서(다음)의 target값
#                 next_pred = next_pred.tolist()
#                 semi_next_list = [next_pred[i] for i in list_of_next_allowed_machines] 
#                 Q_future = max(semi_next_list) #다음 state에서 target 최대 값
            target[0][action_id] = reward + Q_future * self.learning_rate # target의 action_id번째 위치에 다음 Q값이 들어감. 맞춰야 하는 값!!!
      
            
            self.model.fit(state, target, epochs= 1, verbose= 0) #1에폭으로 학습



    # update our target network 
    def train_target(self): #학습에서 가중치를 업데이트
        print("update weights")
        weights = self.model.get_weights() 
        target_weights = self.target_model.get_weights()
        for i in range(len(target_weights)):
            target_weights[i] = weights[i] * self.tau + target_weights[i] * (1 - self.tau)#loss함수?
        self.target_model.set_weights(target_weights)



    # save our model 
    def save_model(self, fn):
        self.model.save(fn)


# Initialize the policy and the target network
dqn_agent = DQN()
# def flowUpdate_interval(env,action,Tfu): #Tsc에서의 action(pacint rate)가 n번의 flow update interval만큼 유지된다.
#     #print("flow update interval :",env.now)
#     global request
#     fs=request['filesize']
#     #src=request['sources']
#     while True:
#         print("flow update interval :",env.now)
#         if ((fs[0]!=0)or(fs[1]!=0)or(fs[2]!=0)): #하나라도 0이상으로 남아있으면
            
#             print("전송중!영차영차")
            
#             for s in range(len(sources)):#0밑으로는 안떨어지게, 전역변수 request의 값을 변경
#                 request['filesize'][s]=max([request['filesize'][s]-action[s],0]) 
#                 request['deadline'][s]=max([request['deadline'][s]-1,0])
#                 if ((request['filesize'][s]==0) and (request['value'][s]==None)): #아직 완료되지 않았던 flow가 전송이 완료되면?
#                     #print ("s{}의 전송이 완료됨".format(s))
#                     if (request['deadline'][s]!=0): #기간 안에 전송되면?
#                         request['value'][s]=1 #value를 1로 변경
#                         print ("s{}의 전송이 deadline 안에 완료됨".format(s))
#                     else: #기간안에 전송된게 아니라면(value초기값은 None)
#                         request['value'][s]=0
#                         print ("s{}의 전송이 deadline을 지나 완료됨".format(s))
#             #print (env.now)
#             yield env.timeout(Tfu) 
#         elif ((fs[0]==0)and(fs[1]==0)and(fs[2]==0)):
#             print ("종료:",env.now)
#             return #모든 flow의 전송이 완료되면 종료 >이걸 다른 함수로 빼야하나?
#     while(i!=lenst): #station의 개수만큼 실행(2*5=10)
#         state = [[m.queue_length for m in s] for s in st]
#         allowed_actions = [m for m in st[i] if m.queue_length < m.max_q_length] #allowed action은 최대 queue length 미만인 machine

#         # appending the current station number to the state
#         state = np.append(state, i)
#         print("current state", state)
#         print(" ")
#         print(" ")
#         dispatch_machine = dqn_agent.choose_action(state, allowed_actions) #action 선택
#         #print("selected action", dispatch_machine.name)
#         dispatch_machine.queue_length = dispatch_machine.queue_length + 1 #선택된 action(machine)의 queue길이 증가
                
        
def scheduling_interval(env,Tsc,Tfu,allowed_actions): #pacing rate가 각 flow에게 할당
    global action
    global state
    global request
    fs=request['filesize']
    c=0
    while ((request['filesize'][0]!=0)or(request['filesize'][1]!=0)or(request['filesize'][2]!=0)):#모두 전송이 완료될 때 까지
        c+=1
        print("")
        print ("Scheduling interval : ",c)
        print("")
        #scheduling interval의 초기에 action을 update하고, 끝날 때 reward를 할당한다.
        #state는 각 source의 min_rate로서, 남은 filesize/남은 deadlin까지의 기한 을 뜻한다.
        #deadline이 0인 경우(이미 초과하거나 완료된 경우) 0으로 나누기 때문에 에러 발생. 즉 deadline이 0이면 이전 state를 할당
        #또한, float형을 int형으로 형변환해버림으로서 반올림이 아니라 버림을 함 -> 따라서 math패키지의 ceil()을 활용해 올림연산
        try: 
            state=[math.ceil(request['filesize'][s]/request['deadline'][s]) for s in range(len(request['sources']))]
        except:#deadline이 0인 경우 error
            for s in range(len(sources)):
                if (request['deadline'][s] != 0):
                    state[s]= math.ceil(request['filesize'][s]/request['deadline'][s])
                else:
                    state[s]=0
            
        #         except:
#             for s in range(len(sources)):
#                 if (request['deadline'][s] != 0):
#                     state[s]= int(request['filesize'][s]/request['deadline'][s]) 
#                 else:
#                     state[s]= 0
                
        print("현재 sc에서의 state",state)
        
#         for i in range(len(sources)):#전송 완료된 source가 있다면 action은 0으로 할당
#             if (request['filesize'][i]==0):
#                 action[i]=0
            
        action=dqn_agent.choose_action(state,allowed_actions) #action이 None으로 return되는 현상?
        print("현재 sc에서의 action",action)
        
        
        
        for i in range(Tsc):#하나의 scheduling interval 내의 flow update interval
            print("flow update interval :",env.now)
            #print("전송중!영차영차")

            for s in range(len(sources)):#0밑으로는 안떨어지게, 전역변수 request의 값을 변경(각source에 대한 반복)
                print ("filesize of source",s)
                print (request['filesize'][s])
                print ("deadline of source",s)
                print (request['deadline'][s])
                request['filesize'][s]=max([request['filesize'][s]-action[s],0]) 
                request['deadline'][s]=max([request['deadline'][s]-1,0])
                if ((request['filesize'][s]==0) and (request['value'][s]==None)): #아직 완료되지 않았던 flow가 전송이 완료되면?
                    #print ("s{}의 전송이 완료됨".format(s))
                    if (request['deadline'][s]!=0): #기간 안에 전송되면?
                        request['value'][s]=1 #value를 1로 변경
                        print ("s{}의 전송이 deadline 안에 완료됨".format(s))
                    else: #기간안에 전송된게 아니라면(value초기값은 None)
                        request['value'][s]=0
                        print ("s{}의 전송이 deadline을 지나 완료됨".format(s))
            yield env.timeout(Tfu) 
            
        next_state=[math.ceil(request['filesize'][s]/request['deadline'][s]) for s in range(len(request['sources'])) if request['deadline'][s] != 0 ]#update된 request상황으로 next_state 정의
        
        # appending the current station number to the state
        next_state = np.append(next_state, c)

        # build your reward function 리워드 방정식 입력
        reward = sum((np.max(request['deadline'])-request['deadline'])*action)

#         if i < lenst-1: #전체 queue를 모두 처리하면 done=True가 됨
#             done = False
#             next_allowed_actions = [m for m in st[i+1] if m.queue_length < m.max_q_length]
#         else:
#             done = True

        cur_state = state 
        action = action
        new_state = next_state 
        #new_action = next_allowed_actions
        reward = reward
        #done = done

        dqn_agent.remember(cur_state, action, reward, new_state) #새로운 state로 설정해주고 기존state저장
        dqn_agent.replay(allowed_actions)#학습

        
        
    return #모든 전송이 끝나면 return(종료)

        
        
        




#한 번의 에피소드 프로세스
#N개의 source에서 동시에 request를 발생시키면 시작됨
#끝나는 기점은 모든 flow가 전송이 완료되는 시점
#랜덤으로 filesize를 생성, deadline도 랜덤으로 생성
env = simpy.Environment()
print("Episode start")
print ("")

bottleneck_capacity=10 #bottleneck link의 capacity는 10Gbps
state=[0,0,0]#초기 state
action=[0,0,0]#초기 action ->고칠것
Tsc=5 #scheduling interval을 구성하는 flow update interval의 수
Tfu=1 #flow update interval의 시간단위(1초로 가정)
sources = ['s0', 's1', 's2']
destinations = ['d0', 'd1', 'd2']
request = {
    'sources' : sources,
    'destinations' : destinations, 
    'filesize' : [random.randrange(100,500) for source in sources], #단위는 Gbps
    'deadline' : [random.randrange(80,150) for source in sources], 
    'value' : [None for source in sources], #아직 전송되지 않았으면 None, 제시간에 전송되었으면 1, 제시간에 전송되지 않았으면 0
}

A=list(range(10))
B=list(range(10))
C=list(range(10)) #합이 10이 되는 0~10까지의 수 조합
allowed_actions=[]
for i in list(product(*(A,B,C))):
    if sum(list(i))==10:
        allowed_actions.append(i) #d는 63개의 조합
        


print ("source 0의 filesize:",request['filesize'][0])
print ("source 0의 deadline:",request['deadline'][0])
print("")
print ("source 1의 filesize:",request['filesize'][1])
print ("source 1의 deadline:",request['deadline'][1])
print("")
print ("source 2의 filesize:",request['filesize'][2])
print ("source 2의 deadline:",request['deadline'][2])



env.process(scheduling_interval(env,Tsc,Tfu,allowed_actions))
env.run()

for i in range(len(sources)):
    if (request['value'][i]==1):
        print ("source {} 전송완료".format(i))
    else:
        print ("source {} deadine 충족하지 못함".format(i))
print("모두 전송 완료")

Episode start

source 0의 filesize: 373
source 0의 deadline: 123

source 1의 filesize: 325
source 1의 deadline: 117

source 2의 filesize: 237
source 2의 deadline: 132

Scheduling interval :  1

현재 sc에서의 state [4, 3, 2]
현재 sc에서의 action (5, 3, 2)
flow update interval : 0
filesize of source 0
373
deadline of source 0
123
filesize of source 1
325
deadline of source 1
117
filesize of source 2
237
deadline of source 2
132
flow update interval : 1
filesize of source 0
368
deadline of source 0
122
filesize of source 1
322
deadline of source 1
116
filesize of source 2
235
deadline of source 2
131
flow update interval : 2
filesize of source 0
363
deadline of source 0
121
filesize of source 1
319
deadline of source 1
115
filesize of source 2
233
deadline of source 2
130
flow update interval : 3
filesize of source 0
358
deadline of source 0
120
filesize of source 1
316
deadline of source 1
114
filesize of source 2
231
deadline of source 2
129
flow update interval : 4
filesize of source 0
353
deadline of

record states
train

Scheduling interval :  18

현재 sc에서의 state [3, 0, 2]
현재 sc에서의 action (2, 7, 1)
flow update interval : 85
filesize of source 0
103
deadline of source 0
38
filesize of source 1
0
deadline of source 1
32
filesize of source 2
52
deadline of source 2
47
flow update interval : 86
filesize of source 0
101
deadline of source 0
37
filesize of source 1
0
deadline of source 1
31
filesize of source 2
51
deadline of source 2
46
flow update interval : 87
filesize of source 0
99
deadline of source 0
36
filesize of source 1
0
deadline of source 1
30
filesize of source 2
50
deadline of source 2
45
flow update interval : 88
filesize of source 0
97
deadline of source 0
35
filesize of source 1
0
deadline of source 1
29
filesize of source 2
49
deadline of source 2
44
flow update interval : 89
filesize of source 0
95
deadline of source 0
34
filesize of source 1
0
deadline of source 1
28
filesize of source 2
48
deadline of source 2
43
record states
train

Scheduling interval :  19

현재 sc에

1.할 것 : flow update interval을 줄이기 (0.001초?) 어느정도가 적당할 지 교수님꼐여줘보기

2.reward의 문제점: drem(deadline까지 남아있는 시간)이 가장 짧은 source에 많은 pacing rate를 할당할 수록 높은 reward가 주어지는데, file size를 고려하지 않았다는 문제점이 있다. deadline이 아니라 min rate를 기준으로 해야할듯

3.state=0인 경우 filesize=0이란뜻이므로 action을 0으로 할당하는 코드를 짜야함. 문제는 하나의 filesize가 0인경우 어떻게 하냐는것...!!!!!

In [100]:
#allowed action 생성
state=[1,2,2]
allowed_actions=[[st for st in range(s,10)] for s in state]
al=list(product(*allowed_actions))
allowedactions=[list(a) for a in al if sum(list(a))<=10]
allowedactions

[[1, 2, 2],
 [1, 2, 3],
 [1, 2, 4],
 [1, 2, 5],
 [1, 2, 6],
 [1, 2, 7],
 [1, 3, 2],
 [1, 3, 3],
 [1, 3, 4],
 [1, 3, 5],
 [1, 3, 6],
 [1, 4, 2],
 [1, 4, 3],
 [1, 4, 4],
 [1, 4, 5],
 [1, 5, 2],
 [1, 5, 3],
 [1, 5, 4],
 [1, 6, 2],
 [1, 6, 3],
 [1, 7, 2],
 [2, 2, 2],
 [2, 2, 3],
 [2, 2, 4],
 [2, 2, 5],
 [2, 2, 6],
 [2, 3, 2],
 [2, 3, 3],
 [2, 3, 4],
 [2, 3, 5],
 [2, 4, 2],
 [2, 4, 3],
 [2, 4, 4],
 [2, 5, 2],
 [2, 5, 3],
 [2, 6, 2],
 [3, 2, 2],
 [3, 2, 3],
 [3, 2, 4],
 [3, 2, 5],
 [3, 3, 2],
 [3, 3, 3],
 [3, 3, 4],
 [3, 4, 2],
 [3, 4, 3],
 [3, 5, 2],
 [4, 2, 2],
 [4, 2, 3],
 [4, 2, 4],
 [4, 3, 2],
 [4, 3, 3],
 [4, 4, 2],
 [5, 2, 2],
 [5, 2, 3],
 [5, 3, 2],
 [6, 2, 2]]

In [82]:
#deeplearning없이 시뮬레이션
env = simpy.Environment()

request = {
    'sources' : sources,
    'destinations' : destinations, 
    'filesize' : [random.randrange(50,100) for source in sources], #단위는 Gbps
    'deadline' : [random.randrange(5,30) for source in sources], 
    'value' : [None for source in sources], #아직 전송되지 않았으면 None, 제시간에 전송되었으면 1, 제시간에 전송되지 않았으면 0
}

print ("source 0의 filesize:",request['filesize'][0])
print ("source 1의 filesize:",request['filesize'][1])
print ("source 2의 filesize:",request['filesize'][2])

action=[3,3,4]

Tfu=1

def flowUpdate_interval(env,action,Tfu): #Tsc에서의 action(pacint rate)가 n번의 flow update interval만큼 유지된다.
    #print("flow update interval :",env.now)
    global request
    fs=request['filesize']
    #src=request['sources']
    while True:
        print("flow update interval :",env.now)
        if ((fs[0]!=0)or(fs[1]!=0)or(fs[2]!=0)): #하나라도 0이상으로 남아있으면
            
            print("전송중!영차영차")
            
            for s in range(len(sources)):#0밑으로는 안떨어지게, 전역변수 request의 값을 변경
                request['filesize'][s]=max([request['filesize'][s]-action[s],0]) 
                request['deadline'][s]=max([request['deadline'][s]-1,0])
                if ((request['filesize'][s]==0) and (request['value'][s]==None)): #아직 완료되지 않았던 flow가 전송이 완료되면?
                    #print ("s{}의 전송이 완료됨".format(s))
                    if (request['deadline'][s]!=0): #기간 안에 전송되면?
                        request['value'][s]=1 #value를 1로 변경
                        print ("s{}의 전송이 deadline 안에 완료됨".format(s))
                    else: #기간안에 전송된게 아니라면(value초기값은 None)
                        request['value'][s]=0
                        print ("s{}의 전송이 deadline을 지나 완료됨".format(s))
            
            yield env.timeout(Tfu) 
        elif ((fs[0]==0)and(fs[1]==0)and(fs[2]==0)):
            print ("종료:",env.now)
            return #모든 flow의 전송이 완료되면 종료
        
        
proc=env.process(flowUpdate_interval(env,action,Tfu))
env.run(until=proc)

source 0의 filesize: 87
source 1의 filesize: 56
source 2의 filesize: 78
flow update interval : 0
전송중!영차영차
flow update interval : 1
전송중!영차영차
flow update interval : 2
전송중!영차영차
flow update interval : 3
전송중!영차영차
flow update interval : 4
전송중!영차영차
flow update interval : 5
전송중!영차영차
flow update interval : 6
전송중!영차영차
flow update interval : 7
전송중!영차영차
flow update interval : 8
전송중!영차영차
flow update interval : 9
전송중!영차영차
flow update interval : 10
전송중!영차영차
flow update interval : 11
전송중!영차영차
flow update interval : 12
전송중!영차영차
flow update interval : 13
전송중!영차영차
flow update interval : 14
전송중!영차영차
flow update interval : 15
전송중!영차영차
flow update interval : 16
전송중!영차영차
flow update interval : 17
전송중!영차영차
flow update interval : 18
전송중!영차영차
s1의 전송이 deadline 안에 완료됨
flow update interval : 19
전송중!영차영차
s2의 전송이 deadline 안에 완료됨
flow update interval : 20
전송중!영차영차
flow update interval : 21
전송중!영차영차
flow update interval : 22
전송중!영차영차
flow update interval : 23
전송중!영차영차
flow update interval : 24
전송중!영차영차
flow update interv

In [56]:
for s in range(len(sources)):
    print ("D")
    request['filesize'][s]=max([request['filesize'][s]-3,0]) 
#                 global request['deadline'][s]=max([request['deadline'][s]-1,0])
#                 if (request['filesize'][s]==0): #전송이 완료되면?

D
D
D


In [48]:
request['filesize'][0]

120

In [24]:
import random
sources = ['s0', 's1', 's2']
destinations = ['d0', 'd1', 'd2']
requests = {
    'sources' : sources,
    'destinations' : destinations, 
    'filesize' : {source: random.randrange(100,150) for source in sources}, #단위는 Gbps
    'deadline' : {source: random.randrange(5,30) for source in sources}, 
    'value' : {source: 0 for source in sources}, #제시간에 전송되었는지
}
fs=requests['filesize']
